In [ ]:
! pip install tweet-preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Data Cleanup

In [ ]:
# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
import preprocessor as p
def clean_text(df):
    #set up punctuations we want to be replaced
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
    REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")
    tempArr = []
    for line in df:
        # send to tweet_processor
        tmpL = p.clean(line)
        # remove puctuation
        tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
        tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
        tempArr.append(tmpL)
    return tempArr

Read Data

In [ ]:
import pandas as pd
import numpy as np
import re
Tweets= pd.read_csv('/content/sample_data/Twitter-Tweets-c.csv',encoding='utf8', engine='python',error_bad_lines=False)
Tweets = Tweets.drop(['id','label'],axis=1)
Tweets_A = Tweets[0:5000]
Tweets_B = Tweets[5000:10000]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
Tweets_A['Tweet-cleaned'] = clean_text(Tweets_A['tweet'].apply(str))
Tweets_B['Tweet-cleaned'] = clean_text(Tweets_B['tweet'].apply(str))
corpus_A = list(Tweets_A['Tweet-cleaned'])
corpus_B = list(Tweets_B['Tweet-cleaned'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stopwords=set(stopwords.words('english'))


def remove_stopwords(corpus):
    output_array=[]
    for sentence in corpus:
        temp_list=[]
        for word in sentence.split():
            if word.lower() not in stopwords:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

corpus_A=remove_stopwords(corpus_A)
corpus_B=remove_stopwords(corpus_B)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
corpus_A[0:5]

['father dysfunctional selfish drags kids dysfunction',
 'thanks credit cant use cause dont offer wheelchair vans pdx',
 'bihday majesty',
 'love u take u time ur',
 'facts guide society']

In [ ]:
corpus_B[0:5]

['holy shit trump person cheering n e new jersey like lied',
 'days hyper helllove storms work day ending earlay',
 'anyone recognize bandit took forks bike &amp bike station tools help pls',
 'govt transformed says lists achievements',
 'dog']

Glove Implementation

In [ ]:
from scipy import spatial
import gensim.downloader as api

model = api.load("glove-twitter-200")

In [ ]:
def preprocess(s):
    return [i.lower() for i in s.split()]

def get_vector(s):
  try:
    return np.sum(np.array([model[i] for i in preprocess(s)]), axis=0)

  except:"word ' ' not in vocabulary"
  pass

In [ ]:
def cosine_similarity(i, j):
     nominator = np.dot(i, j)
    
     a_norm = np.sqrt(np.sum(i**2))
     b_norm = np.sqrt(np.sum(j**2))
    
     denominator = a_norm * b_norm
    
     cosine_similarity = nominator / denominator
     return cosine_similarity

In [ ]:
case_list = []
for i in range(0,5000):
  list = [corpus_A[i],corpus_B[i]]
  case_list.append(list)

In [ ]:
df = pd.DataFrame(case_list, columns=['Sentences_A', 'Sentences_B'])
df['Cosine_similarity']=""

In [ ]:
df

,Sentences_A,Sentences_B,Cosine_similarity
0,father dysfunctional selfish drags kids dysfun...,holy shit trump person cheering n e new jersey...,
1,thanks credit cant use cause dont offer wheelc...,days hyper helllove storms work day ending earlay,
2,bihday majesty,anyone recognize bandit took forks bike &amp b...,
3,love u take u time ur,govt transformed says lists achievements,
4,facts guide society,dog,
...,...,...,...
4995,wasnt attack lgbt community america well may i...,routine whack evening walk,
4996,looking forward catching designer lucky enough...,im dead still happy,
4997,apps help keep &amp,kingdom claimant count rate may previous,
4998,never go bed without wakeup without,rip friend,


In [ ]:
for i, j in df.iterrows():
  a=get_vector(df['Sentences_A'].values[i])
  b=get_vector(df['Sentences_B'].values[i])
  try:
    df['Cosine_similarity'].values[i] = cosine_similarity(a, b)
  except:'float' and 'NoneType'
  pass


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [ ]:
df

,Sentences_A,Sentences_B,Cosine_similarity
0,father dysfunctional selfish drags kids dysfun...,holy shit trump person cheering n e new jersey...,0.610082
1,thanks credit cant use cause dont offer wheelc...,days hyper helllove storms work day ending earlay,
2,bihday majesty,anyone recognize bandit took forks bike &amp b...,
3,love u take u time ur,govt transformed says lists achievements,0.520411
4,facts guide society,dog,0.480086
...,...,...,...
4995,wasnt attack lgbt community america well may i...,routine whack evening walk,0.709571
4996,looking forward catching designer lucky enough...,im dead still happy,0.816771
4997,apps help keep &amp,kingdom claimant count rate may previous,
4998,never go bed without wakeup without,rip friend,0.67144


In [ ]:
df.to_excel("/content/sample_data/Twitter_Tweets_Output - index(0).xlsx") 